## Nelaturi_MahemaReddy_HW3

#### Name: Mahema Reddy Nelaturi 
#### Github Username: mahemareddy
#### USC ID: 2814429305 

### 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download the AReM data 

In [1]:
import os
import csv
import pandas as pd
import numpy as np
import seaborn as sns

from scipy.stats import bootstrap

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
PATH = '../data/AReM'

### (b) Keep datasets 1 and 2 in folders bending1 and bending 2, as well as datasets 1, 2, and 3 in other folders as test data and other datasets as train data

In [4]:
COLUMNS = ["time", "avg_rss12", "var_rss12", "avg_rss13", "var_rss13", "avg_rss23", "var_rss23"]
STATS = ['mean', 'std', 'min', '1st_quartile', 'median', '3rd_quartile', 'max']
ordered_STATS = ['min', 'max', 'mean', 'median', 'std', '1st_quartile', '3rd_quartile']

In [5]:
def split_dataset(path):
    train_files = []
    test_files = []

    for root, dir, file in os.walk(path):
        for f in file:
            if "pdf" not in f:
                fname = os.path.join(root, f)
                if ("bending1" in fname) or ("bending2" in fname):
                    if ("dataset1" in fname) or ("dataset2" in fname):
                        test_files.append(fname)
                    else:
                        train_files.append(fname)
                else:
                    if ("dataset1.csv" in fname) or ("dataset2.csv" in fname) or ("dataset3.csv" in fname):
                        test_files.append(fname)
                    else:
                        if '.DS_Store' not in f:
                            train_files.append(fname)
    return train_files, test_files

In [6]:
train_files, test_files = split_dataset(PATH)
# train_files
# test_files

### (c) Feature Extraction

#### i. Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc).

- Minimum 
- Maximum 
- Mean 
- Median
- Standard Deviation
- First Quartile 
- third Quartile
- Correlation 

#### ii. Extract the time-domain features minimum, maximum, mean, median, stan-dard deviation, first quartile, and third quartile for all of the 6 time series in each instance.

In [7]:
def get_filename(file):
    path = os.path.dirname(file)
    filename = path.split("\\")
    return filename[-1]

In [8]:
def get_stat_features(stats):
    features = []
    for i in range(1, 7):
        for stat in stats:
            features.append(stat + str(i))
    return features

features = get_stat_features(STATS)
ordered_features = get_stat_features(ordered_STATS)

In [9]:
def load_dataset(files):
    data = []
    labels = []

    for file in files:
        labels.append(get_filename(file))
        sniffer = csv.Sniffer()
        separator = sniffer.sniff(open(file).read()).delimiter
    
        if separator == ",":
            df_file = pd.read_csv(file, skiprows= 5, header= None, on_bad_lines= 'skip')
        else:
            df_file = pd.read_csv(file, skiprows= 5, sep= '\\s+', header= None, on_bad_lines= 'skip')
        df_file.columns = COLUMNS

        description = []
        current = df_file[0 : df_file.shape[0]]
        current_description = current.describe().drop("count").drop(columns= "time").T
        description.append(current_description.values.flatten())
        data.append(np.concatenate(description))

    df = pd.DataFrame(data)
    df.columns = features
    df = df.loc[:, ordered_features]
    df["label"] = pd.Series(labels)

    return df

In [10]:
train_data = load_dataset(train_files)
train_data

,min1,max1,mean1,median1,std1,1st_quartile1,3rd_quartile1,min2,max2,mean2,...,1st_quartile5,3rd_quartile5,min6,max6,mean6,median6,std6,1st_quartile6,3rd_quartile6,label
0,36.25,48.00,43.969125,44.50,1.618364,43.310,44.67,0.0,1.50,0.413125,...,20.5000,23.7500,0.0,2.96,0.555312,0.490,0.487826,0.0000,0.830,../data/AReM/bending1
1,37.00,48.00,43.454958,43.25,1.386098,42.500,45.00,0.0,1.58,0.378083,...,22.2500,24.0000,0.0,5.26,0.679646,0.500,0.622534,0.4300,0.870,../data/AReM/bending1
2,33.00,47.75,42.179812,43.50,3.670666,39.150,45.00,0.0,3.00,0.696042,...,30.4575,36.3300,0.0,2.18,0.613521,0.500,0.524317,0.0000,1.000,../data/AReM/bending1
3,33.00,45.75,41.678063,41.75,2.243490,41.330,42.75,0.0,2.83,0.535979,...,28.4575,31.2500,0.0,1.79,0.383292,0.430,0.389164,0.0000,0.500,../data/AReM/bending1
4,35.00,47.40,43.954500,44.33,1.558835,43.000,45.00,0.0,1.70,0.426250,...,35.3625,36.5000,0.0,1.79,0.493292,0.430,0.513506,0.0000,0.940,../data/AReM/bending1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,18.50,44.25,35.752354,36.00,4.614802,33.000,39.33,0.0,12.60,3.328104,...,14.0000,18.0625,0.0,9.39,3.069667,2.770,1.748326,1.7975,4.060,../data/AReM/cycling
65,24.25,45.00,37.177042,36.25,3.581301,34.500,40.25,0.0,8.58,2.374208,...,17.9500,21.7500,0.0,9.34,2.921729,2.500,1.852600,1.5000,3.900,../data/AReM/cycling
66,23.33,43.50,36.248768,36.75,3.824632,33.415,39.25,0.0,9.71,2.737307,...,15.7500,21.0000,0.0,11.15,3.532463,3.110,1.965267,2.1700,4.625,../data/AReM/cycling
67,26.25,44.25,36.957458,36.29,3.434863,34.500,40.25,0.0,8.64,2.420083,...,14.0000,18.2500,0.0,8.34,2.934625,2.525,1.631380,1.6600,4.030,../data/AReM/cycling


In [11]:
test_data = load_dataset(test_files)
test_data

,min1,max1,mean1,median1,std1,1st_quartile1,3rd_quartile1,min2,max2,mean2,...,1st_quartile5,3rd_quartile5,min6,max6,mean6,median6,std6,1st_quartile6,3rd_quartile6,label
0,37.25,45.00,40.624792,40.500,1.476967,39.2500,42.0000,0.0,1.30,0.358604,...,33.0000,36.0000,0.0,1.92,0.570583,0.430,0.582915,0.0000,1.300,../data/AReM/bending1
1,38.00,45.67,42.812812,42.500,1.435550,42.0000,43.6700,0.0,1.22,0.372437,...,32.0000,34.5000,0.0,3.11,0.571083,0.430,0.601010,0.0000,1.300,../data/AReM/bending1
2,19.33,43.50,34.227771,35.500,4.889576,30.5000,37.7500,0.0,14.50,3.995729,...,14.7500,18.6700,0.0,9.74,3.394125,3.100,1.792090,2.1050,4.425,../data/AReM/walking
3,12.50,45.00,33.509729,34.125,4.850923,30.5000,36.7500,0.0,13.05,4.450771,...,14.6275,18.7500,0.0,8.96,3.378479,3.085,1.787360,2.0600,4.440,../data/AReM/walking
4,15.00,46.75,34.660583,35.000,5.315110,31.0000,38.2500,0.0,13.44,4.200896,...,14.2500,18.5000,0.0,8.99,3.244396,3.000,1.630983,2.1200,4.240,../data/AReM/walking
5,12.75,51.00,24.562958,24.250,3.737514,23.1875,26.5000,0.0,6.87,0.590833,...,20.5000,27.0000,0.0,4.97,0.700188,0.500,0.693720,0.4300,0.870,../data/AReM/bending2
6,0.00,42.75,27.464604,28.000,3.583582,25.5000,30.0000,0.0,7.76,0.449708,...,15.0000,20.7500,0.0,6.76,1.122125,0.830,1.012342,0.4700,1.300,../data/AReM/bending2
7,33.33,48.00,44.334729,45.000,2.476940,42.2500,46.5000,0.0,3.90,0.432958,...,9.3300,17.7500,0.0,5.02,0.933000,0.830,0.673609,0.4700,1.250,../data/AReM/standing
8,35.50,46.25,43.174938,43.670,1.989052,42.5000,44.5000,0.0,2.12,0.506583,...,12.7500,16.5000,0.0,5.72,0.911979,0.830,0.666161,0.4700,1.220,../data/AReM/standing
9,32.75,47.00,42.760562,44.500,3.398919,41.3300,45.3725,0.0,3.34,0.486167,...,13.0000,18.5650,0.0,5.73,0.842271,0.710,0.722165,0.4300,1.090,../data/AReM/standing


#### iii. Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.

In [12]:
train_data.describe().loc['std']

min1             8.794295
max1             4.429182
mean1            4.917692
median1          4.956111
std1             1.758670
1st_quartile1    5.731647
3rd_quartile1    4.783645
min2             0.000000
max2             5.147841
mean2            1.600701
median2          1.436960
std2             0.902808
1st_quartile2    0.952201
3rd_quartile2    2.158416
min3             3.053869
max3             4.759853
mean3            3.863097
median3          3.845730
std3             0.995959
1st_quartile3    4.145255
3rd_quartile3    3.946023
min4             0.000000
max4             2.302408
mean4            1.179861
median4          1.150092
std4             0.473576
1st_quartile4    0.842501
3rd_quartile4    1.566564
min5             5.368786
max5             5.449726
mean5            5.120426
median5          5.267414
std5             1.057998
1st_quartile5    5.543882
3rd_quartile5    4.957231
min6             0.051766
max6             2.540166
mean6            1.171401
median6     

In [13]:
test_data.describe().loc['std']

min1             12.136206
max1              4.379342
mean1             6.790086
median1           7.088085
std1              1.869285
1st_quartile1     7.673052
3rd_quartile1     6.385720
min2              0.000000
max2              4.870395
mean2             1.500529
median2           1.345903
std2              0.832472
1st_quartile2     0.941307
3rd_quartile2     2.028482
min3              2.644618
max3              5.393220
mean3             4.588252
median3           4.753522
std3              0.760779
1st_quartile3     4.554121
3rd_quartile3     5.014339
min4              0.000000
max4              1.733937
mean4             1.146338
median4           1.161705
std4              0.409008
1st_quartile4     0.867088
3rd_quartile4     1.541943
min5              8.252947
max5              6.782153
mean5             7.366781
median5           7.438749
std5              0.919274
1st_quartile5     7.823870
3rd_quartile5     7.259433
min6              0.000000
max6              2.505306
m

In [14]:
data_CI = train_data.iloc[:, :-1]
col = list(data_CI.columns)

lows = []
highs = []
for i in col:
    data = (data_CI[i],)
    bootstrap_CI = bootstrap(data, np.std, confidence_level=0.9, random_state= 1, method='basic')
    lows.append(bootstrap_CI.confidence_interval[0])
    highs.append(bootstrap_CI.confidence_interval[1])
CI_table = pd.DataFrame(columns = ['Low_Confidence', 'High_Confidence'])
CI_table['Low_Confidence'] = lows
CI_table['High_Confidence'] = highs

CI_table.index = col
CI_table

,Low_Confidence,High_Confidence
min1,7.502012,9.989366
max1,3.450496,5.536179
mean1,4.377907,5.479814
median1,4.396912,5.542160
std1,1.559253,1.960701
1st_quartile1,5.204934,6.270986
3rd_quartile1,4.005674,5.619483
min2,0.000000,0.000000
max2,4.735969,5.602172
mean2,1.451224,1.786358


#### iv. Use your judgement to select the three most important time-domain features

- Min
- Max
- Third Quartile 

### 2. ISLR 3.7.4

#### (a) Linear train
- we would expect that RSS for linear regression would be higher than the cubic regression.
- because cubic regression has more predictors and hence will fit the model flexibly

#### (b) Linear test
- we would expect that RSS for linear regression would be lower than the cubic regression; for test data, more predictors in the cubic regression will overfit the model.

#### (c) Non linear train
- Here we would expect th eRSS for cubic regression would be less than Linear Regression; more predictors lead to less RSS, the underlying true relationship does not matter. 

#### (d) Non linear test 
- in this case, it depends on how far the relationship is from linear.
- If it is closer to liner, then it will have RSS less than that for cubic
- If it is closer to cubic, then it will have RSS less than that for linear.